---
# Set 1

##### Basics
---

### Challenge 1

Convert hex to base64

In [9]:
from base64 import b64encode

def hex_to_base64(hexstring):
    byte_seq = bytes.fromhex(hexstring)
    return b64encode(byte_seq)

hex_to_base64('49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d').hex()

'535364744947747062477870626d63676557393163694269636d4670626942736157746c4947456763473970633239756233567a494731316332687962323974'

### Challenge 2

**Fixed XOR:**
Write a function that takes two equal-length buffers and produces their XOR combination.

In [10]:
def fixed_xor(byte_seq1, byte_seq2):
    xor = b''
    for i, j in zip(byte_seq1, byte_seq2):
        xor += bytes([i ^ j])
    return xor

def main():
    byte_seq1 = bytes.fromhex('1c0111001f010100061a024b53535009181c')
    byte_seq2 = bytes.fromhex('686974207468652062756c6c277320657965')
    print(fixed_xor(byte_seq1, byte_seq2))
    
main()

b"the kid don't play"


### Challenge 3

Single-byte XOR cipher

In [11]:
def get_score(input_bytes):
    letter_frequencies = {
        'a': .08167, 'b': .01492, 'c': .02782, 'd': .04253,
        'e': .12702, 'f': .02228, 'g': .02015, 'h': .06094,
        'i': .06094, 'j': .00153, 'k': .00772, 'l': .04025,
        'm': .02406, 'n': .06749, 'o': .07507, 'p': .01929,
        'q': .00095, 'r': .05987, 's': .06327, 't': .09056,
        'u': .02758, 'v': .00978, 'w': .02360, 'x': .00150,
        'y': .01974, 'z': .00074, ' ': .13000
    }
    return sum([letter_frequencies.get(chr(byte), 0) for byte in input_bytes.lower()])

def break_sbyte_xor(byte_seq):
    potential_messages = []
    
    for key_value in range(256):  # each ascii character
        message = b''
        
        for byte in byte_seq:
            xor = bytes([key_value ^ byte])
            message += xor
        
        data = {'Message': message, 'Key': key_value, 'Score': get_score(message)}
        potential_messages.append(data)
        
    best_score = sorted(potential_messages, key=lambda x: x['Score'], reverse=True)[0]
    return best_score

break_sbyte_xor(bytes.fromhex('1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736'))

{'Message': b"Cooking MC's like a pound of bacon", 'Key': 88, 'Score': 2.14329}

### Challenge 4

Detect single-character XOR

In [12]:
from urllib.request import urlopen

def detect_sbyte_xor(lines):
    potential_plaintext = []
    
    for line in lines:
        # Use break_sbyte_xor() method from Challenge 3
        potential_plaintext.append(break_sbyte_xor(line))
    
    best_line = sorted(potential_plaintext, key=lambda x: x['Score'], reverse=True)[0]
    return best_line

def main():
    lines = []
    with urlopen('https://cryptopals.com/static/challenge-data/4.txt') as data:
        for line in data:
            lines.append(bytes.fromhex(line.rstrip().decode()))
    print(detect_sbyte_xor(lines))

main()

{'Message': b'Now that the party is jumping\n', 'Key': 53, 'Score': 2.03479}


### Challenge 5

Implement repeating-key XOR (Vignere Cipher)

In [13]:
def repeating_key_xor(message, key):
    message = message
    key = key
    ciphertext = b''
    
    for i in range(len(message)):
        m = message[i]  # current byte from message
        k = key[i % len(key)]  # byte of key for XOR
        
        xor = bytes([m ^ k])
        ciphertext += xor
    
    return ciphertext

repeating_key_xor(b"Burning 'em, if you ain't quick and nimble I go crazy when I hear a cymbal", b"ICE").hex()

'0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272a282b2f20690a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f'

### Challenge 6

Break repeating-key XOR (Vigenere Cipher)

In [14]:
from urllib.request import urlopen
import numpy as np
from itertools import combinations
from base64 import b64decode

def hamming(block1, block2):
    """Returns the Hamming distance between two equal length byte strings"""
    distance = 0
    for byte1, byte2 in zip(block1, block2):
        byte1 = bin(byte1)[2:].zfill(8).encode()
        byte2 = bin(byte2)[2:].zfill(8).encode()
        distance += sum([1 for bit1, bit2 in zip(byte1, byte2) if bit1 != bit2])
    return distance

def break_repeating_key_xor(ciphertext):
    # Find the keysize, taking advantage of the fact that the Hamming distance between
    # Engligh characters will, on average, be less than the distance between random bytes.
    avg_norm_hd = []  # Averaged normilized Hamming distance
    for keysize in range(2, 41):
        blocks = [ciphertext[i:i+keysize] for i in range(0, len(ciphertext), keysize)]  # break message into keysize-sized blocks
        norm_hd = []  # Normalized Hamming distance
        
        for block1, block2 in combinations(blocks, 2):  # compute the Hamming distance between each block
            norm_hd.append(hamming(block1, block2) / keysize)
        avg_norm_hd.append({'Keysize': keysize, 'Score': np.mean(norm_hd)})
        
    avg_norm_hd = sorted(avg_norm_hd, key=lambda x: x['Score'])[:3]  # Top three results for the keysize
    
    # For each of the top 3 keysizes, transpose the blocks (i.e. chunk ciphertext into blocks with multiples of keysize, then keysize + 1, etc...).
    # Then solve each block with single byte xor, and combine the resulting keys to form the potential key for that keysize.
    # The score for the final key will be the score for all the single byte keys added together. Rank the final keys, and decrypt using the best one.
    possible_key_list = []
    for item in avg_norm_hd:
        keysize = item['Keysize']
        possible_key = b''
        key_score = 0
        
        # Transpose the blocks
        for i in range(keysize):
            block = b''
            for j in range(i, len(ciphertext), keysize):
                block += bytes([ciphertext[j]])
            
            key_info = break_sbyte_xor(block)  # Use break_sbyte_xor() method from Challenge 3
            possible_key += chr(key_info['Key']).encode()  # add single byte key to potential key
            key_score += key_info['Score']  # add score for single byte key to the score for the potential key
        
        possible_key_list.append({'Key': possible_key, 'Score': key_score})
    
    # Sort potential keys, and decrypt using the best one
    best_key = sorted(possible_key_list, key=lambda x: x['Score'], reverse=True)[0]['Key']
    message = repeating_key_xor(ciphertext, best_key)  # Use repeating_key_xor() method from Challenge 5
    return {'Message': message, 'Key': best_key}

def main():
    # Load message, removing new line characters along the way
    ciphertext = b''
    with urlopen('https://cryptopals.com/static/challenge-data/6.txt') as data:
        for line in data:
            ciphertext += b64decode(line.rstrip())
    print(break_repeating_key_xor(ciphertext))

main()

{'Message': b"I'm back and I'm ringin' the bell \nA rockin' on the mike while the fly girls yell \nIn ecstasy in the back of me \nWell that's my DJ Deshay cuttin' all them Z's \nHittin' hard and the girlies goin' crazy \nVanilla's on the mike, man I'm not lazy. \n\nI'm lettin' my drug kick in \nIt controls my mouth and I begin \nTo just let it flow, let my concepts go \nMy posse's to the side yellin', Go Vanilla Go! \n\nSmooth 'cause that's the way I will be \nAnd if you don't give a damn, then \nWhy you starin' at me \nSo get off 'cause I control the stage \nThere's no dissin' allowed \nI'm in my own phase \nThe girlies sa y they love me and that is ok \nAnd I can dance better than any kid n' play \n\nStage 2 -- Yea the one ya' wanna listen to \nIt's off my head so let the beat play through \nSo I can funk it up and make it sound good \n1-2-3 Yo -- Knock on some wood \nFor good luck, I like my rhymes atrocious \nSupercalafragilisticexpialidocious \nI'm an effect and that you can bet \

### Challenge 7

AES in ECB mode

In [15]:
from urllib.request import urlopen
from base64 import b64decode
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend

def AES_128_ECB_encrypt(plaintext, key):
    cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(plaintext) + encryptor.finalize()
    return ciphertext

def AES_128_ECB_decrypt(ciphertext, key):
    cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=default_backend())
    decryptor = cipher.decryptor()
    plaintext = decryptor.update(ciphertext) + decryptor.finalize()
    return plaintext

def main():
    ciphertext = b''
    with urlopen('https://cryptopals.com/static/challenge-data/7.txt') as data:
        for line in data:
            ciphertext += b64decode(line.rstrip())
    print(AES_128_ECB_decrypt(ciphertext, b'YELLOW SUBMARINE'))

main()

b"I'm back and I'm ringin' the bell \nA rockin' on the mike while the fly girls yell \nIn ecstasy in the back of me \nWell that's my DJ Deshay cuttin' all them Z's \nHittin' hard and the girlies goin' crazy \nVanilla's on the mike, man I'm not lazy. \n\nI'm lettin' my drug kick in \nIt controls my mouth and I begin \nTo just let it flow, let my concepts go \nMy posse's to the side yellin', Go Vanilla Go! \n\nSmooth 'cause that's the way I will be \nAnd if you don't give a damn, then \nWhy you starin' at me \nSo get off 'cause I control the stage \nThere's no dissin' allowed \nI'm in my own phase \nThe girlies sa y they love me and that is ok \nAnd I can dance better than any kid n' play \n\nStage 2 -- Yea the one ya' wanna listen to \nIt's off my head so let the beat play through \nSo I can funk it up and make it sound good \n1-2-3 Yo -- Knock on some wood \nFor good luck, I like my rhymes atrocious \nSupercalafragilisticexpialidocious \nI'm an effect and that you can bet \nI can take 

### Challenge 8

Detect AES in ECB mode

In [16]:
from urllib.request import urlopen

def detect_AES_ECB(ciphertext, keysize=16):
    blocks = [ciphertext[i:i+keysize] for i in range(0, len(ciphertext), keysize)]
    return len(blocks) - len(set(blocks))

def main():
    scores = []
    ciphertext_list = []
    with urlopen('https://cryptopals.com/static/challenge-data/8.txt') as data:
        for line in data:
            ciphertext = bytes.fromhex(line.rstrip().decode())
            scores.append(detect_AES_ECB(ciphertext))
            ciphertext_list.append(ciphertext)
    
    best_score = max(scores)
    print(f"ECB mode detected on line {scores.index(best_score) + 1}\nNumber of repetitions = {best_score}")
    print()
    print(ciphertext_list[scores.index(best_score)].hex())
    
main()

ECB mode detected on line 133
Number of repetitions = 3

d880619740a8a19b7840a8a31c810a3d08649af70dc06f4fd5d2d69c744cd283e2dd052f6b641dbf9d11b0348542bb5708649af70dc06f4fd5d2d69c744cd2839475c9dfdbc1d46597949d9c7e82bf5a08649af70dc06f4fd5d2d69c744cd28397a93eab8d6aecd566489154789a6b0308649af70dc06f4fd5d2d69c744cd283d403180c98c8f6db1f2a3f9c4040deb0ab51b29933f2c123c58386b06fba186a


---
# Set 2

##### Block Crypto
---

### Challenge 9

Implement PKCS#7 padding

In [70]:
"""Superseded by Challenge 15"""

def PKCS7_pad(plaintext, block_size):
    pad_size = block_size - (len(plaintext) % block_size)
    assert 0 < pad_size < 256
    padding = b''.join([bytes([pad_size] * pad_size)])
    return plaintext + padding

def PKCS7_unpad(padded_data):
    pad_size = padded_data[-1]
    unpadded_data = padded_data[:-pad_size]
    return unpadded_data

def main():
    data = b'YELLOW SUBMARINE'
    padded_data = PKCS7_pad(data, 20)
    unpadded_data = PKCS7_unpad(padded_data)
    print(f'{data} -> {padded_data} -> {unpadded_data}')

main()

b'YELLOW SUBMARINE' -> b'YELLOW SUBMARINE\x04\x04\x04\x04' -> b'YELLOW SUBMARINE'


In [10]:
# The following is a test of the AES_ECB encrypt and decrypt methods.

# message = PKCS7_pad(
#     b'It was the best of times, it was the worst of times, '
#     b'it was the age of wisdom, it was the age of foolishness, '
#     b'it was the epoch of belief, it was the epoch of incredulity, '
#     b'it was the season of Light, it was the season of Darkness, '
#     b'it was the spring of hope, it was the winter of despair.', 16
# )
# key = b'YELLOW SUBMARINE'
# ciphertext = AES_128_ECB_encrypt(message, key)
# plaintext = PKCS7_unpad(AES_128_ECB_decrypt(ciphertext, key))
# print(plaintext)

b'It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of foolishness, it was the epoch of belief, it was the epoch of incredulity, it was the season of Light, it was the season of Darkness, it was the spring of hope, it was the winter of despair.'


### Challenge 10

Implement CBC mode

In [18]:
import os
from urllib.request import urlopen

def AES_128_CBC_encrypt(plaintext, key, iv):
    # Key must be 128, 192, or 256 bits, and plaintext must already be padded
    assert len(key) == 16 or len(key) == 24 or len(key) == 32
    assert len(plaintext) % 16 == 0
    
    blocks = [plaintext[i:i+16] for i in range(0, len(plaintext), 16)]
    ciphertext_blocks = []
    
    for block in blocks:
        if len(ciphertext_blocks) == 0:
            xor_block = fixed_xor(block, iv)  # fixed_xor() from Challenge 2
        else:
            xor_block = fixed_xor(block, ciphertext_blocks[-1])  # fixed_xor() from Challenge 2
        
        ciphertext_blocks.append(AES_128_ECB_encrypt(xor_block, key))  # ECB encryption from Challenge 7
    
    ciphertext = b''.join(ciphertext_blocks)
    return ciphertext

def AES_128_CBC_decrypt(ciphertext, key, iv):
    # Key must be 128, 192, or 256 bits, and ciphertext must already be padded
    assert len(key) == 16 or len(key) == 24 or len(key) == 32
    assert len(ciphertext) % 16 == 0
    
    blocks = [ciphertext[i:i+16] for i in range(0, len(ciphertext), 16)]
    plaintext_blocks = []
    
    for i in range(len(blocks)):
        block = blocks[i]
        
        decrypted_block = AES_128_ECB_decrypt(block, key)  # ECB decryption from Challenge 7
        
        if len(plaintext_blocks) == 0:
            plaintext_blocks.append(fixed_xor(decrypted_block, iv))  # fixed_xor() from Challenge 2
        else:
            plaintext_blocks.append(fixed_xor(decrypted_block, blocks[i - 1]))  # fixed_xor() from Challenge 2
    
    plaintext = b''.join(plaintext_blocks)
    return plaintext
            
def main():
    key = b'YELLOW SUBMARINE'
    iv = os.urandom(16)
    message = PKCS7_pad(  # Padding function from Challenge 9
        b'It was the best of times, it was the worst of times, it was the age of wisdom, '
        b'it was the age of foolishness, it was the epoch of belief, it was the epoch of incredulity, '
        b'it was the season of Light, it was the season of Darkness, it was the spring of hope, '
        b'it was the winter of despair.', 16
    )
    ciphertext = AES_128_CBC_encrypt(message, key, iv)
    plaintext = PKCS7_unpad(AES_128_CBC_decrypt(ciphertext, key, iv))  # Padding function from Challenge 9
    print(plaintext)
    
#     key = b'YELLOW SUBMARINE'
#     iv = bytes(16)
#     ciphertext = b''
#     with urlopen('https://cryptopals.com/static/challenge-data/10.txt') as data:
#         for line in data:
#             ciphertext += b64decode(line.rstrip())
    
#     plaintext = PKCS7_unpad(AES_128_CBC_decrypt(ciphertext, key, iv))  # Padding function from Challenge 9
#     print(plaintext)
    
main()

b'It was the best of times, it was the worst of times, it was the age of wisdom, it was the age of foolishness, it was the epoch of belief, it was the epoch of incredulity, it was the season of Light, it was the season of Darkness, it was the spring of hope, it was the winter of despair.'


### Challenge 11

An ECB/CBC detection oracle

In [19]:
import os
from random import randint

def encryption_oracle(plaintext):
    rand_key = os.urandom(16)
    front_pad = os.urandom(randint(5, 10))
    back_pad = os.urandom(randint(5, 10))
    data = PKCS7_pad(front_pad + plaintext + back_pad, 16)
    if randint(0, 1):
        return AES_128_ECB_encrypt(data, rand_key), 'ECB'
    else:
        return AES_128_CBC_encrypt(data, rand_key, iv=os.urandom(16)), 'CBC'

def detect_ECB_or_CBC(ciphertext, plaintext_length):
    score = detect_AES_ECB(ciphertext)  # from Challenge 8
    
    # If at least half of the blocks are duplicates, then we guess ECB mode.
    # The 50% requirement is somewhat arbitrary, but is based on Challenge 12.
    # For my code here, you can increase the requirement to 66% before it
    # starts making mistakes
    if (score / (plaintext_length // 16)) * 100 >= 50:
        return 'ECB'
    else:
        return 'CBC'

def main():
    percentage = 0
    r = 10000
    for _ in range(r):
        message = b'A' * randint(100, 200)
        ciphertext, mode = encryption_oracle(message)
        potential_mode = detect_ECB_or_CBC(ciphertext, len(message))
        if mode == potential_mode:
            percentage += 1
    print(f"The mode was correct {(percentage / r) * 100}% of the time.")

main()

The mode was correct 100.0% of the time.


### Challenge 12

Byte-at-a-time ECB decryption (Simple)

In [20]:
import os
from base64 import b64decode

rand_key = os.urandom(16)
target_string = b64decode(
    b'Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkg' +
    b'aGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBq' +
    b'dXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUg' +
    b'YnkK'
)

def encryption_oracle(plaintext):
    plaintext = PKCS7_pad(plaintext + target_string, 16)
    return AES_128_ECB_encrypt(plaintext, rand_key)

def get_block_and_pad_size():
    """
    Starts with encrypting no plaintext, so just the prefix and target string.
    Then add one byte of input at a time until the padding causes the output
    to jump up by one block size. The difference between the new output length
    and the original output length is the block size, and the number of bytes
    added to the input is the pad size.
    """
    base_length = len(encryption_oracle(bytes()))
    pad_size = 1
    while True:
        data = b'A' * pad_size
        new_length = len(encryption_oracle(data))
        potential_block_size = new_length - base_length
        if potential_block_size:
            return potential_block_size, pad_size
        pad_size += 1

def match_next_byte(input_string, output, start, end):
    """
    We start out knowing all but one of the bytes in the output block. So we try
    all 256 possibilities for the last byte, and return the one that creates
    the same block when encrypted with the input we already know.
    """
    for c in range(256):
        guess_byte = chr(c).encode()
        guess_input = input_string + guess_byte
        guess_output = encryption_oracle(guess_input)[start:end]
        if guess_output == output:
            return guess_byte
    raise Exception("No match found")

def get_target_string(block_size, pad_size):
    """
    If we know the input for a whole block except the last byte, it's pretty easy
    to figure out what that last byte is, since ECB is deterministic. So we input
    enough bytes so that there is a block where the last byte of the block is the
    first byte of target_string, and all the others are our input. Then we find
    the first byte of target_string. Then we remove one byte from our input, so
    that same block now has two bytes from target_string, but now we know what the
    first byte is, so we can solve for the second byte of target_string. Repeat
    until we have done that for the whole string.
    """
    base_length = len(encryption_oracle(bytes()))
    solved_string = b''
    for i in range(base_length-1, pad_size-1, -1):
        input_string = (b'A' * i)
        
        start = base_length - block_size
        end = base_length
        output = encryption_oracle(input_string)[start:end]
        
        solved_string += match_next_byte(input_string+solved_string, output, start, end)
    return solved_string

def main():
    block_size, pad_size = get_block_and_pad_size()
    assert detect_ECB_or_CBC(encryption_oracle(b'A' * 32), 32) == 'ECB'
    solved_string = get_target_string(block_size, pad_size)
    print(solved_string)

main()

b"Rollin' in my 5.0\nWith my rag-top down so my hair can blow\nThe girlies on standby waving just to say hi\nDid you stop? No, I just drove by\n"


### Challenge 13

ECB cut-and-paste

In [21]:
from uuid import uuid4
import os

rand_key = os.urandom(16)

def kv_parsing(data_in):
    data_out = {}
    for item in data_in.split('&'):
        item = item.split('=')
        if item[0] == 'email':
            item[1] = item[1].replace('%26', '&').replace('%3D', '=')
        data_out[item[0]] = item[1]
    return data_out

def profile_for(email):
    email = email.replace('&', '%26').replace('=', '%3D')
    profile = f'email={email}&uid={uuid4()}&role=user'
    return encrypt_profile(profile)

def encrypt_profile(profile):
    plaintext = PKCS7_pad(profile.encode(), 16)
    return AES_128_ECB_encrypt(plaintext, rand_key)

def decrypt_profile(ciphertext):
    plaintext = PKCS7_unpad(AES_128_ECB_decrypt(ciphertext, rand_key))
    return kv_parsing(plaintext.decode())

def main():
    """
    The email address is a specific length so that the term 'user',
    along with the padding, is all that appears in the last block of the
    encrypted profile. Then I replace that last block with the block you would
    get from encrypting the term 'admin'. Boom! Admin privlages!
    """
    email_domain = "@gmail.com"
    initial_bytes = len(f'email={email_domain}&uid={uuid4()}&role=')
    email_length = 16 - (initial_bytes % 16)
    email = 'A' * email_length + email_domain
    
    profile = profile_for(email)[:-16]
    profile += AES_128_ECB_encrypt(PKCS7_pad(b'admin', 16), rand_key)
    profile = decrypt_profile(profile)
    print("The role of this profile is '{}'".format(profile['role']))

    # Funny enough, even without the escaped characters, this attack doesn't work.
    # It does set the role to admin, but this is immediately overwritten when it
    # parses the role=user part of the profile string. Because this comes after
    # the user input, the attacker has no way to change this. They can, however,
    # create new fields or alter existing ones that do not get changed later on.
#     profile = profile_for('malicious&role=admin')
#     print()
#     profile = decrypt_profile(profile)
#     print("The role of this profile is '{}'".format(profile['role']))

main()

The role of this profile is 'admin'


### Challenge 14

Byte-at-a-time ECB decryption (Harder)

In [25]:
import os
from random import randint
from base64 import b64decode

rand_key = os.urandom(16)
rand_prefix = os.urandom(randint(0, 256))
target_string = b64decode(
    b'Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkg' +
    b'aGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBq' +
    b'dXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUg' +
    b'YnkK'
)

def encryption_oracle(plaintext):
    plaintext = PKCS7_pad(rand_prefix + plaintext + target_string, 16)
    return AES_128_ECB_encrypt(plaintext, rand_key)

def get_block_and_pad_size():
    """
    Starts with encrypting no plaintext, so just the prefix and target string.
    Then add one byte of input at a time until the padding causes the output
    to jump up by one block size. The difference between the new output length
    and the original output length is the block size, and the number of bytes
    added to the input is the pad size.
    """
    base_length = len(encryption_oracle(bytes()))
    pad_size = 1
    while True:
        data = b'A' * pad_size
        new_length = len(encryption_oracle(data))
        potential_block_size = new_length - base_length
        if potential_block_size:
            return potential_block_size, pad_size
        pad_size += 1

def get_input_start(block_size):
    """
    Input the bytes 0-30 (one less than two full blocks) to the encryption_oracle.
    This guarantees that there will be exactly one full block in the output that
    is just a part of that sequence. By finding where that block is, and where in
    the sequence it started, we know where the prefix ends.
    """
    input_string = bytes([i for i in range((2 * block_size) - 1)])
    output = encryption_oracle(input_string)
    
    known_blocks = []
    for i in range(16):
        input_block = input_string[i:i+block_size]
        known_blocks.append(AES_128_ECB_encrypt(input_block, rand_key))
    
    for i in range(0, len(output), block_size):
        current_block = output[i:i+block_size]
        for known_block in known_blocks:
            if current_block == known_block:
                return i - known_blocks.index(known_block)

def match_next_byte(input_string, output, start, end):
    """
    We start out knowing all but one of the bytes in the output block. So we try
    all 256 possibilities for the last byte, and return the one that creates
    the same block when encrypted with the input we already know.
    """
    for c in range(256):
        guess_byte = chr(c).encode()
        guess_input = input_string + guess_byte
        guess_output = encryption_oracle(guess_input)[start:end]
        if guess_output == output:
            return guess_byte
    raise Exception("No match found")

def get_target_string(block_size, pad_size, input_start, target_len):
    """
    If we know the input for a whole block except the last byte, it's pretty easy
    to figure out what that last byte is, since ECB is deterministic. So we input
    enough bytes so that there is a block where the last byte of the block is the
    first byte of target_string, and all the others are our input. Then we find
    the first byte of target_string. Then we remove one byte from our input, so
    that same block now has two bytes from target_string, but now we know what the
    first byte is, so we can solve for the second byte of target_string. Repeat
    until we have done that for the whole string.
    """
    base_length = target_len + pad_size
    solved_string = b''
    for i in range(base_length-1, pad_size-1, -1):
        input_string = (b'A' * i)
        
        start = input_start + base_length - block_size
        end = start + block_size
        output = encryption_oracle(input_string)[start:end]
        
        solved_string += match_next_byte(input_string+solved_string, output, start, end)
    return solved_string

def main():
    block_size, pad_size = get_block_and_pad_size()
    input_start = get_input_start(block_size)
    target_len = len(encryption_oracle(bytes())) - pad_size - input_start
    solved_string = get_target_string(block_size, pad_size, input_start, target_len)
    print(solved_string)

main()

b"Rollin' in my 5.0\nWith my rag-top down so my hair can blow\nThe girlies on standby waving just to say hi\nDid you stop? No, I just drove by\n"


### Challenge 15

PKCS#7 padding validation

In [71]:
class PKCS7PaddingException(Exception):
    """Handles invalid padding in the unpad method"""
    pass


def PKCS7_pad(plaintext, block_size):
    pad_size = block_size - (len(plaintext) % block_size)
    assert 0 < pad_size < 256
    padding = b''.join([bytes([pad_size] * pad_size)])
    return plaintext + padding

def PKCS7_validate(padded_data, pad_size):
    """
    Raises an exception if:
        - pad_size is longer than padded_data
        - padding slice is not uniform
    """
    if pad_size > len(padded_data):
        raise PKCS7PaddingException('pad_size larger than the length of the data')
    padding = padded_data[-pad_size:]
    for byte in padding:
        if byte != pad_size:
            raise PKCS7PaddingException('padding not uniform')
    return True

def PKCS7_unpad(padded_data):
    pad_size = padded_data[-1]
    PKCS7_validate(padded_data, pad_size)  # raise exception if padding is invalid
    unpadded_data = padded_data[:-pad_size]
    return unpadded_data

def main():
    data = b'YELLOW SUBMARINE'
    padded_data = PKCS7_pad(data, 20)
#     padded_data = bytes(15) + bytes([10])
    unpadded_data = PKCS7_unpad(padded_data)
    print(f'{data} -> {padded_data} -> {unpadded_data}')

main()

b'YELLOW SUBMARINE' -> b'YELLOW SUBMARINE\x04\x04\x04\x04' -> b'YELLOW SUBMARINE'


### Challenge 16

CBC bitflipping attacks

In [27]:
import os

rand_key = os.urandom(16)
iv = os.urandom(16)

def encryption_oracle(plaintext):
    plaintext = plaintext.replace(b';', b'%3B').replace(b'=', b'%3D')
    prefix = b"comment1=cooking%20MCs;userdata="
    suffix = b";comment2=%20like%20a%20pound%20of%20bacon"
    input_string = PKCS7_pad(prefix + plaintext + suffix, 16)
    return AES_128_CBC_encrypt(input_string, rand_key, iv)

def is_admin(ciphertext):
    plaintext = PKCS7_unpad(AES_128_CBC_decrypt(ciphertext, rand_key, iv))
    if b';admin=true;' in plaintext:
        return True
    else:
        return False

def main():
    ciphertext = encryption_oracle(os.urandom(16) + b'AadminAtrueA')  # 0, 6, 11
    ciphertext = bytearray(ciphertext)
    
    for i in [0, 6, 11]:
        if i == 0 or i == 11:
            to_byte = ord(';')
        else:
            to_byte = ord('=')
            
        ciphertext[32+i] = ciphertext[32+i] ^ ord('A') ^ to_byte
        
    ciphertext = bytes(ciphertext)
    
    print(f'admin={is_admin(ciphertext)}')

main()

admin=True


---
# Set 3

##### Block & Stream Crypto
---

### Challenge 17

The CBC padding oracle

In [117]:
import os
from random import choice, randint

input_strings = [
    b'MDAwMDAwTm93IHRoYXQgdGhlIHBhcnR5IGlzIGp1bXBpbmc=',
    b'MDAwMDAxV2l0aCB0aGUgYmFzcyBraWNrZWQgaW4gYW5kIHRoZSBWZWdhJ3MgYXJlIHB1bXBpbic=',
    b'MDAwMDAyUXVpY2sgdG8gdGhlIHBvaW50LCB0byB0aGUgcG9pbnQsIG5vIGZha2luZw==',
    b'MDAwMDAzQ29va2luZyBNQydzIGxpa2UgYSBwb3VuZCBvZiBiYWNvbg==',
    b'MDAwMDA0QnVybmluZyAnZW0sIGlmIHlvdSBhaW4ndCBxdWljayBhbmQgbmltYmxl',
    b'MDAwMDA1SSBnbyBjcmF6eSB3aGVuIEkgaGVhciBhIGN5bWJhbA==',
    b'MDAwMDA2QW5kIGEgaGlnaCBoYXQgd2l0aCBhIHNvdXBlZCB1cCB0ZW1wbw==',
    b'MDAwMDA3SSdtIG9uIGEgcm9sbCwgaXQncyB0aW1lIHRvIGdvIHNvbG8=',
    b'MDAwMDA4b2xsaW4nIGluIG15IGZpdmUgcG9pbnQgb2g=',
    b'MDAwMDA5aXRoIG15IHJhZy10b3AgZG93biBzbyBteSBoYWlyIGNhbiBibG93'
]

rand_key = os.urandom(16)

def encryption_oracle(plaintext):
#     plaintext = PKCS7_pad(choice(input_strings), 16)
    plaintext = PKCS7_pad(plaintext, 16)
    iv = os.urandom(16)
    return AES_128_CBC_encrypt(plaintext, rand_key, iv), iv

def padding_oracle(ciphertext, iv):
    plaintext = AES_128_CBC_decrypt(ciphertext, rand_key, iv)
    try:
        PKCS7_unpad(plaintext)
    except PKCS7PaddingException:
        return False
    else:
        return True

def decrypt_block(block, iv):
    """
    Modify iv to recover the plaintext corresponding to block
    """
    iv_const = iv  # original version of iv
    iv = bytearray(iv)  # so we can edit particular elements
    
    plaintext = b''
    start_guess = 0  # in case we need to redo a step
    
    while len(plaintext) < 16:
        pad_size = len(plaintext) + 1
        
        # Add the padding for the bytes we already know
        for i in range(1, pad_size):
            iv[-i] = iv_const[-i] ^ plaintext[-i] ^ pad_size
        
        for guess in range(start_guess, 256):
            # Make a guess at the next byte
            iv[-pad_size] = iv_const[-pad_size] ^ guess ^ pad_size
            
            # If the padding is valid, then we probably know the next byte
            if padding_oracle(block, iv):
                plaintext = bytes([guess]) + plaintext
                start_guess = 0
                break
                
        # If no match is found, then the last byte recovered was wrong
        else:
            start_guess = plaintext[0] + 1  # start where we left off last time
            plaintext = plaintext[1:]  # remove the last byte recovered
    
    return plaintext

def decrypt_message(ciphertext, iv):
    plaintext = b''
    blocks = [iv]
    blocks.extend([ciphertext[i:i+16] for i in range(0, len(ciphertext), 16)])
    
    for i in range(1, len(blocks)):
        plaintext += decrypt_block(blocks[i], blocks[i-1])
    
    return PKCS7_unpad(plaintext)

def main():
#     failures = 0
#     for _ in range(100):
#         message = os.urandom(randint(0, 256))
#         ciphertext, iv = encryption_oracle(message)
#         plaintext = decrypt_message(ciphertext, iv)
#         if plaintext != message:
#             failures += 1
#             print("Uh oh, something didn't work...\n")
#             print(message)
#             print()
#             print(plaintext)
#             print('\n' + '-' * 30 + '\n')
#     if failures == 0:
#         print("All messages successfully decrypted!")
        
    ciphertext, iv = encryption_oracle(choice(input_strings))
    plaintext = decrypt_message(ciphertext, iv)
    
    if plaintext in input_strings:
        print("Message Decrypted!")
    else:
        print("Uh oh, something didn't work...")

main()

Message Decrypted!


### Challenge 18

Implement CTR, the stream cipher mode

In [107]:
from struct import pack
from random import randint
from base64 import b64decode

def get_keystream_block(key, nonce, counter):
    block = pack('<Q', nonce) + pack('<Q', counter)
    return AES_128_ECB_encrypt(block, key)

def keystream_generator(key, nonce, counter):
    while True:
        keystream_block = get_keystream_block(key, nonce, counter)
        for byte in keystream_block:
            yield byte
        counter += 1

def AES_128_CTR(message, key, nonce=0, counter=0):
    """Both the excryption and decryption method"""
    output = b''
    keystream = keystream_generator(key, nonce, counter)
    for byte in message:
        output += bytes([byte ^ next(keystream)])
    return output

def main():
    ciphertext = b64decode(b'L77na/nrFsKvynd6HzOoG7GHTLXsTVu9qvY/2syLXzhPweyyMTJULu/6/kXX0KSvoOLSFQ==')
    plaintext = AES_128_CTR(ciphertext, b'YELLOW SUBMARINE')
    print(plaintext)

main()

b"Yo, VIP Let's kick it Ice, Ice, baby Ice, Ice, baby "


### Challenge 19

Break fixed-nonce CTR mode using substitutions

(apparently I solved this pretty much the way I was supposed to solve challenge 20, but whatever)

In [255]:
import os
from base64 import b64decode

input_strings = [
    b'SSBoYXZlIG1ldCB0aGVtIGF0IGNsb3NlIG9mIGRheQ==',
    b'Q29taW5nIHdpdGggdml2aWQgZmFjZXM=',
    b'RnJvbSBjb3VudGVyIG9yIGRlc2sgYW1vbmcgZ3JleQ==',
    b'RWlnaHRlZW50aC1jZW50dXJ5IGhvdXNlcy4=',
    b'SSBoYXZlIHBhc3NlZCB3aXRoIGEgbm9kIG9mIHRoZSBoZWFk',
    b'T3IgcG9saXRlIG1lYW5pbmdsZXNzIHdvcmRzLA==',
    b'T3IgaGF2ZSBsaW5nZXJlZCBhd2hpbGUgYW5kIHNhaWQ=',
    b'UG9saXRlIG1lYW5pbmdsZXNzIHdvcmRzLA==',
    b'QW5kIHRob3VnaHQgYmVmb3JlIEkgaGFkIGRvbmU=',
    b'T2YgYSBtb2NraW5nIHRhbGUgb3IgYSBnaWJl',
    b'VG8gcGxlYXNlIGEgY29tcGFuaW9u',
    b'QXJvdW5kIHRoZSBmaXJlIGF0IHRoZSBjbHViLA==',
    b'QmVpbmcgY2VydGFpbiB0aGF0IHRoZXkgYW5kIEk=',
    b'QnV0IGxpdmVkIHdoZXJlIG1vdGxleSBpcyB3b3JuOg==',
    b'QWxsIGNoYW5nZWQsIGNoYW5nZWQgdXR0ZXJseTo=',
    b'QSB0ZXJyaWJsZSBiZWF1dHkgaXMgYm9ybi4=',
    b'VGhhdCB3b21hbidzIGRheXMgd2VyZSBzcGVudA==',
    b'SW4gaWdub3JhbnQgZ29vZCB3aWxsLA==',
    b'SGVyIG5pZ2h0cyBpbiBhcmd1bWVudA==',
    b'VW50aWwgaGVyIHZvaWNlIGdyZXcgc2hyaWxsLg==',
    b'V2hhdCB2b2ljZSBtb3JlIHN3ZWV0IHRoYW4gaGVycw==',
    b'V2hlbiB5b3VuZyBhbmQgYmVhdXRpZnVsLA==',
    b'U2hlIHJvZGUgdG8gaGFycmllcnM/',
    b'VGhpcyBtYW4gaGFkIGtlcHQgYSBzY2hvb2w=',
    b'QW5kIHJvZGUgb3VyIHdpbmdlZCBob3JzZS4=',
    b'VGhpcyBvdGhlciBoaXMgaGVscGVyIGFuZCBmcmllbmQ=',
    b'V2FzIGNvbWluZyBpbnRvIGhpcyBmb3JjZTs=',
    b'SGUgbWlnaHQgaGF2ZSB3b24gZmFtZSBpbiB0aGUgZW5kLA==',
    b'U28gc2Vuc2l0aXZlIGhpcyBuYXR1cmUgc2VlbWVkLA==',
    b'U28gZGFyaW5nIGFuZCBzd2VldCBoaXMgdGhvdWdodC4=',
    b'VGhpcyBvdGhlciBtYW4gSSBoYWQgZHJlYW1lZA==',
    b'QSBkcnVua2VuLCB2YWluLWdsb3Jpb3VzIGxvdXQu',
    b'SGUgaGFkIGRvbmUgbW9zdCBiaXR0ZXIgd3Jvbmc=',
    b'VG8gc29tZSB3aG8gYXJlIG5lYXIgbXkgaGVhcnQs',
    b'WWV0IEkgbnVtYmVyIGhpbSBpbiB0aGUgc29uZzs=',
    b'SGUsIHRvbywgaGFzIHJlc2lnbmVkIGhpcyBwYXJ0',
    b'SW4gdGhlIGNhc3VhbCBjb21lZHk7',
    b'SGUsIHRvbywgaGFzIGJlZW4gY2hhbmdlZCBpbiBoaXMgdHVybiw=',
    b'VHJhbnNmb3JtZWQgdXR0ZXJseTo=',
    b'QSB0ZXJyaWJsZSBiZWF1dHkgaXMgYm9ybi4='
]

def get_score(byte):
    letter_frequencies = {
        'a': .08167, 'b': .01492, 'c': .02782, 'd': .04253,
        'e': .12702, 'f': .02228, 'g': .02015, 'h': .06094,
        'i': .06094, 'j': .00153, 'k': .00772, 'l': .04025,
        'm': .02406, 'n': .06749, 'o': .07507, 'p': .01929,
        'q': .00095, 'r': .05987, 's': .06327, 't': .09056,
        'u': .02758, 'v': .00978, 'w': .02360, 'x': .00150,
        'y': .01974, 'z': .00074, ' ': .13000
    }
    return letter_frequencies.get(chr(byte).lower(), 0)

def get_keystream_byte(data):
    scores = []
    for guess in range(256):
        score = 0
        for byte in data:
            score += get_score(byte ^ guess)
        
        scores.append({'Byte': bytes([guess]), 'Score': score})
    
    best_guess = sorted(scores, key=lambda x: x['Score'], reverse=True)[0]['Byte']
    return best_guess

def main():
    rand_key = os.urandom(16)
    ciphertexts = []
    for line in input_strings:
        ciphertexts.append(AES_128_CTR(b64decode(line), rand_key))
    
    keystream = b''
    for i in range(max(len(line) for line in ciphertexts)):
        data = []
        for j in range(len(ciphertexts)):
            # Handles lines longer than the shortest line
            try:
                data.append(ciphertexts[j][i])
            except IndexError:
                pass
        keystream += get_keystream_byte(data)
    
    plaintexts = []
    for line in ciphertexts:
        plaintexts.append(fixed_xor(line, keystream))
    
    # Because the ends of the longer lines have fewer data points to compare, they
    # have a tendancy to be wrong. This could be fixed with a lot of effort
    # matching bigrams, trigrams, etc..., but I don't want to. So deal with it
    for line in plaintexts:
        print(line.decode())
    
main()

I have met them at close of day
Coming with vivid faces
From counter or desk among grey
Eighteenth-century houses.
I have passed with a nod of theht E
Or polite meaningless words,
Or have lingered awhile and saiE
Polite meaningless words,
And thought before I had done
Of a mocking tale or a gibe
To please a companion
Around the fire at the club,
Being certain that they and I
But lived where motley is worn:
All changed, changed utterly:
A terrible beauty is born.
That woman's days were spent
In ignorant good will,
Her nights in argument
Until her voice grew shrill.
What voice more sweet than hers
When young and beautiful,
She rode to harriers?
This man had kept a school
And rode our winged horse.
This other his helper and frienE
Was coming into his force;
He might have won fame in the eOd=
So sensitive his nature seemed,
So daring and sweet his thought
This other man I had dreamed
A drunken, vain-glorious lout.
He had done most bitter wrong
To some who are near my heart,
Yet I number 

### Challenge 20

Break fixed-nonce CTR statistically (just a rehash of how I solved challenge 19)

In [279]:
from urllib.request import urlopen
from base64 import b64decode
import os

def get_score(byte):
    letter_frequencies = {
        'a': .08167, 'b': .01492, 'c': .02782, 'd': .04253,
        'e': .12702, 'f': .02228, 'g': .02015, 'h': .06094,
        'i': .06094, 'j': .00153, 'k': .00772, 'l': .04025,
        'm': .02406, 'n': .06749, 'o': .07507, 'p': .01929,
        'q': .00095, 'r': .05987, 's': .06327, 't': .09056,
        'u': .02758, 'v': .00978, 'w': .02360, 'x': .00150,
        'y': .01974, 'z': .00074, ' ': .13000
    }
    return letter_frequencies.get(chr(byte).lower(), 0)

def get_keystream_byte(data, rank):
    scores = []
    for guess in range(256):
        score = 0
        for byte in data:
            score += get_score(byte ^ guess)
        
        scores.append({'Byte': bytes([guess]), 'Score': score})
    
    best_guess = sorted(scores, key=lambda x: x['Score'], reverse=True)[rank]
    return best_guess['Byte']

def main():
    rand_key = os.urandom(16)
    ciphertexts = []
    with urlopen("https://cryptopals.com/static/challenge-data/20.txt") as data:
        for line in data:
            plaintext = b64decode(line.rstrip())
            ciphertexts.append(AES_128_CTR(plaintext, rand_key))
    
    keystream = b''
    for i in range(max(len(line) for line in ciphertexts)):
        data = []
        for j in range(len(ciphertexts)):
            # Handles lines longer than the shortest line
            try:
                data.append(ciphertexts[j][i])
            except IndexError:
                pass
        # The correct first keystream byte is actually the third or fourth best
        # scoring byte (the other one gives the same letters, but lowercase), so
        # this corrects for that. Not a code issue, just a random coincidence.
        if i == 0:
            rank = 2
        else:
            rank = 0
        keystream += get_keystream_byte(data, rank)
    
    plaintexts = []
    for line in ciphertexts:
        plaintexts.append(fixed_xor(line, keystream))
    
    # Because the ends of the longer lines have fewer data points to compare, they
    # have a tendancy to be wrong. This could be fixed with a lot of effort
    # matching bigrams, trigrams, etc..., but I don't want to. So deal with it
    for line in plaintexts:
        print(line.decode())
    
#     print()
    
#     for line in ciphertexts:
#         print(AES_128_CTR(line, rand_key))

main()

I'm rated "R"...this is a warning, ya better void / Poets are paranoid, DJ's D-stroyed
Cuz I came back to attack others in spite- / Strike like lightnin', It's quite frightenin'!
But don't be afraid in the dark, in a park / Not a scream or a cry, or a bark, more like a spark
Ya tremble like a alcoholic, muscles tighten up / What's that, lighten up! You see a sight but
Suddenly you feel like your in a horror flick / You grab your heart then wish for tomorrow quick
Music's the clue, when I come your warned / Apocalypse Now, when I'm done, ya gone!
Haven't you ever heard of a MC-murderer? / This is the death penalty,and I'm servin' a
Death wish, so come on, step to this / Hysterical idea for a lyrical professionist!
Friday the thirteenth, walking down Elm Street / You come in my realm ya get beat!
This is off limits, so your visions are blurry / All ya see is the meters at a volume
Terror in the styles, never error-files / Indeed I'm known-your exiled!
For those that oppose to be level 

### Challenge 21

Implement the MT19937 Mersenne Twister RNG

In [ ]:
class MT19937:
    def __init__(self):
        